# **Big Data Analysis- Proyecto Big-Data**

Integrantes: Daniel Fernando Silva Avila - 20201395008.

# ENMARCAR LA PREOCUPACIÓN
1. Identificación de los interesados: Se encuentran directamente interesados los directivos del comercio electrónico, principalmente los responsables del sistema de distribución de productos.
2. Pregunta inicial.
*¿El producto llegará tarde a su destino?*

Esta es la generalización de la pregunta de negocio con respuesta binaria (si o no).

3. Análisis de por qué y para qué.
* ¿Por qué?: Se han identificado falencias en el sistema de distribución de mercancías (entregas tardías) que generan quejas y reclamos por parte del cliente que adquiere mercancía a través de nuestra plataforma de comercio electrónico. Estas quejas se traducen finalmente en perdidas económicas (los clientes se rehúsan a usar nuevamente el servicio y comparte experiencias negativas en redes sociales).
* ¿Para qué?: Con el objetivo de mejorar el proceso de distribución y prometer al cliente un tiempo de entrega que sea cumplido por nuestra organización.
* Objetivos del análisis: Construir e implementar un modelo de predicción de la estimación del riesgo de entrega tardía.

# Objetivo de negocio: 
Reducir el numero de entregas tardias (esto implica reducir tambien reducir el numero de quejas y reclamos por tal motivo)
# Objetivos especificos del notebook

-Desarrollar análisis exploratorio de datos EDA en el dataset elegido para el proyecto usando la libreria de python Pandas-Profiling, el análisis debe incluir las conclusiones de las operaciones de revisión efectuadas con cada libreria.

-Basado en el análisis exploratorio, realizar el proceso de limpieza de datos

-Realizar análisis de dos o más variables y crear graficas de visualización con las principales librerias dispuesta para tal fin en python

-Plantear hipotesis  nulas y alternativas que sean rechazadas o aceptadas basados en metodos de estadistica inferencial

-Construir algoritmos de clasificación que permitan estimar el riesgo de entrega tardia

# Dataset
El conjunto de datos contiene información de la cadena de suministro para un comercio electrónico. Los datos describen procesos de aprovisionamiento, producción, ventas y distribución comercial. También permite la correlación de Datos Estructurados con Datos No Estructurados para la generación de conocimiento. El dataset  ha sido tomado del repositorio kaggle y se encuentra disponible a través del siguiente enlace: https://www.kaggle.com/shashwatwork/dataco-smart-supply-chain-for-big-data-analysis



In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling as pp # exploratory data analysis EDA
import matplotlib.pyplot as plt # data visualisation
import seaborn as sns #data visualisation
import plotly.express as px #data visualisation
import plotly.graph_objects as go
from scipy.stats import chi2_contingency, norm # Calculo de chi2

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Leer el dataset que ha sido importado directamente desde kaggle 

In [ ]:
df = pd.read_csv("../input/dataco-smart-supply-chain-for-big-data-analysis/DataCoSupplyChainDataset.csv", engine='python')

Explorar los primeros 5 registros del dataset

In [ ]:
df.head()

Crear reporte de Pandas profile

In [ ]:
profile = pp.ProfileReport(df, title = "EDA")

In [ ]:
# profile.to_notebook_iframe() # Genera el reporte directamente en el notebook.

Guardar el reporte como un archivo con extensión html

In [ ]:
profile.to_file(output_file="reporte.html") 

**Datos nulos**
Se encontraron datos nulos en tres columnas, Order Zipcode ( 86% valores nulos, es decir un total de 155679 registros), Product description (100% de valores nulos, es decir un total de 180519 registros) y customer zipcode (<0.1% es decir un total de 3 registros), en conjunto estas celdas vacias corresponden al 3.5% del total del dataset

**Variables categorcas**
Existen 28 variables que han sido identificadas como tipo categorico, no obstante, algunas variables se clasificaron como categoricas cuando son numericas, por ejemplo Days for shipping y Order item quantity, esto sucede debido a que todos los valores se agrupan en 4 varoles discretos, lo que genera una confusión por parte de pandas profiling. Con el fin de efectuar análisis estas variables deben ser consideradas como numericas

**Variables numericas**
Existen 24 variables  que han sido identificadas como tipo numerico

**Conclusiones EDA**
No incluir en el análisis las columnas order zipcode (casi vacia), product description (totalmente vacias), customer email (constante sin información), Customer password (Constante sin información), adicionalmente podemos reemplazar los valores nulos de la columna customer zipcode. 
Finalmente existen algunas  columnas que pueden obviarse a la hora de hacer el análisis ya que no aportan información relevante para el objeto del estudio, dichas columnas son: Product Status,Customer Street,Customer Fname,Customer Lname,Latitude,Longitude,Product Image.

# Data Cleaning

In [ ]:
# shape and data types of the data
print(df.shape)
print(df.dtypes)
 
    # se puede revisar la clasificación de pandas profiling contra el tipo de dato 

Eliminaremos las columnas innecesarias

In [ ]:
data=df.drop(['Order Zipcode','Product Description', 'Customer Email','Customer Password','Product Status','Customer Street','Customer Fname','Customer Lname',
           'Latitude','Longitude','Product Image',],axis=1)
data.shape

Los valores nulos de zipcode seran reemplazados por el valor mas común

In [ ]:
column_index=data.columns.get_loc("Customer Zipcode")
# Get the index of the column "Customer Zipcode"
data['Customer Zipcode']=data['Customer Zipcode'].fillna(data.mode().iloc[column_index])
#Filling NaN columns with most common value

# Descriptive analysis and visualisation

Se elabora una matriz de correlación

In [ ]:
fig, ax = plt.subplots(figsize=(24,12))         # figsize
sns.heatmap(data.corr(),annot=True,linewidths=.5,fmt='.1g',cmap= sns.diverging_palette(230, 20, as_cmap=True)) # Heatmap for correlation matrix


Se elaboran las tablas de frecuencia relativa para variables categoricas

In [ ]:
count=data['Delivery Status'].value_counts()  #change categoric variable
print(count / len(data))

¿Cual es la categoria que más se entrega de forma tardia?

In [ ]:
#Filtering columns with late delivery status
late_delivery = data[(data['Delivery Status'] == 'Late delivery')]
#Top 10 products with most late deliveries
fig = px.bar(late_delivery['Category Name'].value_counts().nlargest(10), 
             title="Top 10 products with most late deliveries",
            labels={'value':'Number of late deliveries','index':'Category'})
fig.show()

No obstante, la grafica anterior representa la situación real del comercio electronico debido a que no se tiene en cuenta que el datset se encuentra desbalanceado, por lo que la mayor cantidad de registros se agrupan en unas pocas categorias, al tener en cuenta la proporcionalidad (porcentage de entregas tardias respecto al total de entregas de dicha categoria) se obtiene una grafica diferente que muestra mejor la situación real

In [ ]:
#Calculating proproptional late deliveries
late_count=late_delivery['Category Name'].value_counts()
total_count=data['Category Name'].value_counts()
proportional_count=late_count/total_count*100
fig = px.bar(proportional_count.nlargest(15), 
             title="Top 10 products with highest rate of late delivery",
            labels={'value':'Percentage of late deliveries','index':'Category'})
fig.show()


Con esta grafica se puede deducir que no existe relación entre la categoria del producto y la entrega tardia, y que todas las categorias tienen aproximadamente un 50 % de entregas tardias

¿Cual es el numero de entregas tardias de acuerdo con el tipo de envío y la región?
Para esta grafica se realizan dos versiones usando dos librerias diferentes de python con el fin de comparar sus resultados, en la primera se crea una grafica estatica usando matplotlib mientas que la segunda es una grafica dinamica creada usando plotly

In [ ]:
#Filtering late delivery orders with standard class shipping
xyz1 = data[(data['Delivery Status'] == 'Late delivery') & (data['Shipping Mode'] == 'Standard Class')]
#Filtering late delivery orders with first class shipping
xyz2 = data[(data['Delivery Status'] == 'Late delivery') & (data['Shipping Mode'] == 'First Class')]
#Filtering late delivery orders with second class shipping
xyz3 = data[(data['Delivery Status'] == 'Late delivery') & (data['Shipping Mode'] == 'Second Class')]
#Filtering late delivery orders with same day shipping
xyz4 = data[(data['Delivery Status'] == 'Late delivery') & (data['Shipping Mode'] == 'Same Day')]
#Counting total values
count1=xyz1['Order Region'].value_counts()
count2=xyz2['Order Region'].value_counts()
count3=xyz3['Order Region'].value_counts()
count4=xyz4['Order Region'].value_counts()
#Index names
names=data['Order Region'].value_counts().keys()
n_groups=23
fig,ax = plt.subplots(figsize=(20,8))
index=np.arange(n_groups)
bar_width=0.2
opacity=0.6
type1=plt.bar(index,count1,bar_width,alpha=opacity,color='b',label='Standard Class')
type2=plt.bar(index+bar_width,count2,bar_width,alpha=opacity,color='r',label='First class')
type3=plt.bar(index+bar_width+bar_width,count3,bar_width,alpha=opacity,color='g',label='second class')
type4=plt.bar(index+bar_width+bar_width+bar_width,count4,bar_width,alpha=opacity,color='y',label='same day')
plt.xlabel('Order Regions')
plt.ylabel('Number of shipments')
plt.title('Different Types of shipping methods used in all regions')
plt.legend()
plt.xticks(index+bar_width,names,rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
#Filtering late delivery orders with standard class shipping
xyz1 = data[(data['Delivery Status'] == 'Late delivery') & (data['Shipping Mode'] == 'Standard Class')]
#Filtering late delivery orders with first class shipping
xyz2 = data[(data['Delivery Status'] == 'Late delivery') & (data['Shipping Mode'] == 'First Class')]
#Filtering late delivery orders with second class shipping
xyz3 = data[(data['Delivery Status'] == 'Late delivery') & (data['Shipping Mode'] == 'Second Class')]
#Filtering late delivery orders with same day shipping
xyz4 = data[(data['Delivery Status'] == 'Late delivery') & (data['Shipping Mode'] == 'Same Day')]
#Counting total values
count1=xyz1['Order Region'].value_counts()
count2=xyz2['Order Region'].value_counts()
count3=xyz3['Order Region'].value_counts()
count4=xyz4['Order Region'].value_counts()
#Index names
names=data['Order Region'].value_counts().keys()

fig = go.Figure()
fig.add_trace(go.Bar(x=names,
                y=count1,
                name='Standard Class',
                marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=names,
                y=count2,
                name='First Class',
                marker_color='rgb(26, 118, 255)'
                ))
fig.add_trace(go.Bar(x=names,
                y=count3,
                name='Second Class',
                marker_color='rgb(100, 231, 186)'
                ))
fig.add_trace(go.Bar(x=names,
                y=count4,
                name='Same Day',
                marker_color='rgb(243, 134, 59)'
                ))

fig.update_layout(
    title='Different Types of shipping methods used in all regions',
    xaxis={'categoryorder':'total descending'},
    yaxis=dict(
        title='Number of shipments',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=1,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()


# **Estadistica inferencial**

Prueba de hipotesis

*Hipotesis Nula H0*:

1. La region a la cual se realiza el envío no influye en la entrega tardia

2. El tipo de envío no influye en la entrega tardia

3. La categoria del producto no influye e la entrega tardia

*Hipotesis Alternativa Ha*: 

1. La region a la cual se realiza el envío  influye en la entrega tardia

2. El tipo de envío  influye en la entrega tardia

3. La categoria del producto  influye e la entrega tardia

In [ ]:
def calcular_chi2(dependiente,independientes):
    for var in independientes:
        primary_location_cross = pd.crosstab(data[dependiente], data[var])
        g, p, dof, expctd = chi2_contingency(primary_location_cross)
        print("p-value de Chi-square test para " + dependiente + " vs " + var + " = " , p)


columnas = ['Order Region','Shipping Mode','Category Name','Type','Customer City']

calcular_chi2('Delivery Status', columnas)



Basados en la anterior prueba de hipotesis en la cual se utiliza la distribución Chi2 y se calcularon los niveles de significancia entre
las distintas variables categoricas se puede concluir lo siguiente:

1. Rechazar la hipotesis nula 1
2. Rechazar la hipotesis nula 2
3. Aceptar la hipotesis nula 3

# Algoritmos de CLASIFICACIÓN

*El primer algoritmo a utilizar es regresion logistica

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [ ]:
# Select the columns needed for the model
prediction_data=data[['Order Region','Shipping Mode','Delivery Status']]
prediction_data.columns=['Order_Region','Shipping_Mode','Delivery_Status'] #al incluir la columna Type el modelo baja su precision, al incluir Customer City el modelo no converge
prediction_data=prediction_data[prediction_data.Delivery_Status!='Shipping canceled']
prediction_data=prediction_data[prediction_data.Delivery_Status!='Advance shipping']
prediction_data.head()

In [ ]:
# Feature enginering, one hot encoding
#usar one hot encoding cuando la variable categorica es NOMINAL
prediction_data=pd.get_dummies(prediction_data, drop_first=True)
prediction_data.head()

In [ ]:
#Test train split
X_train, X_test, Y_train, Y_test=train_test_split(prediction_data.drop('Delivery_Status_Shipping on time',axis=1),prediction_data['Delivery_Status_Shipping on time'])

In [ ]:
#train the model
LogReg=LogisticRegression()
LogReg.fit(X_train, Y_train)

In [ ]:
#score the model
LogReg.score(X_test, Y_test)

In [ ]:
Y_pred=LogReg.predict(X_test)
print (classification_report(Y_test,Y_pred))

El modelo creado usando regresion logistica tiene un 75.59% de exactitud

*El segundo algoritmo a utilizar es Random Forest 

In [ ]:
#Import dependencies and train the model
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train,Y_train)

In [ ]:
#score the model
classifier.score(X_test,Y_test)

In [ ]:
Y_pred_RF=classifier.predict(X_test)
print (classification_report(Y_test,Y_pred_RF))

El modelo creado usando el algoritmo de Random Forest tiene un 75.76% de exactitud

**Referencias**
https://www.kaggle.com/skloveyyp/comparison-of-classification-regression-rnn

https://towardsdatascience.com/data-cleaning-in-python-the-ultimate-guide-2020-c63b88bf0a0d

https://towardsdatascience.com/log-book-guide-to-hypothesis-testing-802b1980d0b8